In [1]:
# BertTopic using a pretrained model
from bertopic import BERTopic
from bertopic.vectorizers import ClassTfidfTransformer
from bertopic.representation import KeyBERTInspired
from sentence_transformers import SentenceTransformer
import json
import os
from nltk.tokenize import sent_tokenize

! if not nbformat:
!    pip install --upgrade nbformat==4.2.0

/home/coco/git/lesswrong-alignment-trends/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


fish: Missing end to balance this if statement
 if not nbformat:
 ^^


## Preprocessing

In [2]:
with open(os.path.join(os.getcwd(), "./../lesswrong_data.json")) as f:
    json_data = json.load(f)["posts"]

preprocessed_corpus = []
preprocessed_tags = []

# using the entite post as document: 
for post in json_data:
    preprocessed_corpus.append(post["text"])
    preprocessed_tags.append(post["tags"])

print(preprocessed_tags)
print(len(preprocessed_corpus), len(preprocessed_tags))

[['AI'], ['AI'], ['Interpretability (ML & AI)', 'Language Models (LLMs)', 'AI'], ['AI'], ['AI Governance', 'AI Timelines', 'AI'], ['AI Governance', 'Careers', 'AI'], ['OpenAI', 'Recursive Self-Improvement', 'Reinforcement learning', 'Superintelligence', 'AI'], ['AI'], ['AI'], ['Cognitive Science', 'Forecasting & Prediction', 'Utility Functions', 'AI', 'Rationality'], ['AI'], ['Newsletters', 'AI'], ['AI'], ['AI Control', 'AI'], ['AI Alignment Fieldbuilding', 'Philanthropy / Grant making (Topic)', 'AI'], ['Humor', 'Journalism', 'AI'], ['Simulator Theory', 'AI'], ['AI Control', 'DeepMind', 'Economic Consequences of AGI', 'Financial Investing', 'AI'], ['AI Safety Public Materials', 'AI'], ['Newsletters', 'AI'], ['AI Governance', 'AI'], ['Outer Alignment', 'Aligned AI Proposals', 'Gradient Descent', 'Reinforcement learning', 'AI-Assisted Alignment', 'Inner Alignment', 'Value Learning', 'AI'], ['AI'], ['AI'], ['AI Evaluations', 'Introspection', 'Language Models (LLMs)', 'Situational Awarenes

In [3]:
# get unique tags
unique_tags = []
for tags in preprocessed_tags:
    for tag in tags:
        if tag not in unique_tags:
            unique_tags.append(tag)
print(unique_tags)

['AI', 'Interpretability (ML & AI)', 'Language Models (LLMs)', 'AI Governance', 'AI Timelines', 'Careers', 'OpenAI', 'Recursive Self-Improvement', 'Reinforcement learning', 'Superintelligence', 'Cognitive Science', 'Forecasting & Prediction', 'Utility Functions', 'Rationality', 'Newsletters', 'AI Control', 'AI Alignment Fieldbuilding', 'Philanthropy / Grant making (Topic)', 'Humor', 'Journalism', 'Simulator Theory', 'DeepMind', 'Economic Consequences of AGI', 'Financial Investing', 'AI Safety Public Materials', 'Outer Alignment', 'Aligned AI Proposals', 'Gradient Descent', 'AI-Assisted Alignment', 'Inner Alignment', 'Value Learning', 'AI Evaluations', 'Introspection', 'Situational Awareness', 'AI Alignment Intro Materials', 'Orthogonality Thesis', 'Goal-Directedness', 'Category theory', 'Fixed Point Theorems', 'Formal Proof', 'Gödelian Logic', "Löb's theorem", 'Logic & Mathematics ', 'Logical Uncertainty', 'World Modeling', 'AI Takeoff', 'Consciousness', 'Neuroscience', 'Predictive Pro

In [4]:
import re

def preprocess_text(text):
    # Step 1: Remove LaTeX syntax
    # Remove inline math (e.g., $...$)
    text = re.sub(r'\$.*?\$', '[MATH]', text)
    # Remove LaTeX environments (e.g., \begin{...}...\end{...})
    text = re.sub(r'\\begin\{.*?\}.*?\\end\{.*?\}', '[MATH]', text, flags=re.DOTALL)
    # Remove LaTeX commands (e.g., \frac, \alpha)
    text = re.sub(r'\\[a-zA-Z]+', '', text)
    # Remove mjx and other residual artifacts
    text = re.sub(r'mjx', '', text)


    # Step 2: Normalize text
    text = text.lower().strip()
    # text = re.sub(r'\s+', ' ', text)  # Remove extra spaces

    return text

preprocessed_corpus = [preprocess_text(doc) for doc in preprocessed_corpus]

### Supervised Finetuning of the sentence transformer

In [5]:
preprocessed_corpus

["the new scorecard is on my website,\xa0ai lab watch. this replaces my old scorecard. i redid the content from scratch; it's now up-to-date and higher-quality. i'm also happy with the scorecard's structure: you can click on rows, columns, and cells and zoom in to various things.\xa0check it out! thanks to lightcone for designing the site.\nwhile it is a scorecard, i don't feel great about the numbers; i mostly see it as a collection of information.",
 'i\'m making a website on ai companies\' model evals for dangerous capabilities:\xa0ai safety claims analysis. this is approximately the only analysis of companies\' model evals, as far as i know. this site is in beta; i expect to add lots more content and improve the design in june. i\'ll add content on evals, but i also tentatively plan to expand from evals to evals and safeguards and safety cases (especially now that\xa0a company has said its safeguards are load-bearing for safety!).\nsome cherry-picked bad stuff i noticed when i read

In [6]:

%pip install datasets accelerate>=0.26.0
from sentence_transformers import SentenceTransformer, losses, SentenceTransformerTrainer
from transformers import TrainingArguments, Trainer
from datasets import Dataset

Note: you may need to restart the kernel to use updated packages.


In [8]:
# Step 1: Load a pretrained model
model = SentenceTransformer('all-MiniLM-L6-v2')

# Step 3: Define a Dataset and Loss
dataset = Dataset.from_dict({
    "anchor": preprocessed_corpus,
    "positive": [tags[0] for tags in preprocessed_tags],
})

train_loss = losses.MultipleNegativesRankingLoss(model)

trainer = SentenceTransformerTrainer(
    model=model,
    train_dataset=dataset,
    loss=train_loss,
)

# Step 5: Train the model
trainer.train()

Step,Training Loss


TrainOutput(global_step=78, training_loss=1.0018273378029847, metrics={'train_runtime': 322.226, 'train_samples_per_second': 1.927, 'train_steps_per_second': 0.242, 'total_flos': 0.0, 'train_loss': 1.0018273378029847, 'epoch': 3.0})

In [9]:
# Step 6: Save the fine-tuned model
trainer.save_model("fine_tuned_model")

# Step 7: Use the fine-tuned model in BERTopic
from bertopic import BERTopic
topic_model = BERTopic(embedding_model="fine_tuned_model")

In [10]:
# embedding_model = SentenceTransformer("all-MiniLM-L6-v2")
ctfidf_model = ClassTfidfTransformer(
    seed_words=unique_tags,
    seed_multiplier=5,
    reduce_frequent_words=True
)
representation_model = KeyBERTInspired()

topic_model = BERTopic(
    embedding_model="fine_tuned_model",
    representation_model=representation_model,
    min_topic_size=2,
    nr_topics="none",
    ctfidf_model=ctfidf_model,
    n_gram_range=(1, 5),
    verbose=True
)

topics, probs = topic_model.fit_transform(preprocessed_corpus)


2025-06-18 18:06:04,369 - BERTopic - Embedding - Transforming documents to embeddings.


Batches: 100%|██████████| 7/7 [00:21<00:00,  3.04s/it]
2025-06-18 18:06:26,299 - BERTopic - Embedding - Completed ✓
2025-06-18 18:06:26,301 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
/home/coco/git/lesswrong-alignment-trends/venv/lib/python3.10/site-packages/numba/np/ufunc/parallel.py:371: NumbaWarning: The TBB threading layer requires TBB version 2021 update 6 or later i.e., TBB_INTERFACE_VERSION >= 12060. Found TBB_INTERFACE_VERSION = 12050. The TBB threading layer is disabled.
  warnings.warn(problem)
2025-06-18 18:06:49,178 - BERTopic - Dimensionality - Completed ✓
2025-06-18 18:06:49,182 - BERTopic - Cluster - Start clustering the reduced embeddings
2025-06-18 18:06:49,254 - BERTopic - Cluster - Completed ✓
2025-06-18 18:06:49,257 - BERTopic - Representation - Extracting topics using c-TF-IDF for topic reduction.
2025-06-18 18:06:59,094 - BERTopic - Representation - Completed ✓
2025-06-18 18:06:59,109 - BERTopic - Topic reduction - Reducing number

In [11]:
hierarchical_topics = topic_model.hierarchical_topics(preprocessed_corpus)
topic_model.visualize_hierarchy(hierarchical_topics=hierarchical_topics)

100%|██████████| 19/19 [01:56<00:00,  6.11s/it]


ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'hoverinfo': 'text',
              'marker': {'color': 'rgb(61,153,112)'},
              'mode': 'lines',
              'text': [life_living_life but_civilisation_of life, , , compliance
                       costs_the senate_senate_regulations_senators],
              'type': 'scatter',
              'x': {'bdata': 'AAAAAAAAAACCEASQLyzvP4IQBJAvLO8/AAAAAAAAAAA=', 'dtype': 'f8'},
              'xaxis': 'x',
              'y': {'bdata': 'AAAAAAAAFMAAAAAAAAAUwAAAAAAAAC7AAAAAAAAALsA=', 'dtype': 'f8'},
              'yaxis': 'y'},
             {'hoverinfo': 'text',
              'marker': {'color': 'rgb(61,153,112)'},
              'mode': 'lines',
              'text': [opus_claude opus_asl_claude_as, , , control_control
                       over_agi_ai_robots],
              'type': 'scatter',
              'x': {'bdata': 'AAAAAAAAAAAnIfLER3ztPych8sRHfO0/AAAAAAAAAAA=', 'dtype': 'f8'},
              'xaxis': 'x',
              'y': {'bdata': 'AAAAAACARsAAAAAAAIBGwAAAAAAAgEvAAAAAAACAS8A=', 'dtype': 'f8'},
              'yaxis': 'y'},
             {'hoverinfo': 'text',
              'marker': {'color': 'rgb(61,153,112)'},
              'mode': 'lines',
              'text': [features_networks_input_rank_things, , , opus_claude
                       opus_asl_claude_as],
              'type': 'scatter',
              'x': {'bdata': 'AAAAAAAAAABqczIJduntP2pzMgl26e0/JyHyxEd87T8=', 'dtype': 'f8'},
              'xaxis': 'x',
              'y': {'bdata': 'AAAAAACAQcAAAAAAAIBBwAAAAAAAAEnAAAAAAAAAScA=', 'dtype': 'f8'},
              'yaxis': 'y'},
             {'hoverinfo': 'text',
              'marker': {'color': 'rgb(61,153,112)'},
              'mode': 'lines',
              'text': [ai governance_governance_compute
                       governance_policymaking_policymakers, , , advocacy
                       efforts_advocacy_ai safety advocacy_alignment_safety
                       advocacy],
              'type': 'scatter',
              'x': {'bdata': 'AAAAAAAAAADacnRXZNrtP9pydFdk2u0/AAAAAAAAAAA=', 'dtype': 'f8'},
              'xaxis': 'x',
              'y': {'bdata': 'AAAAAABAUMAAAAAAAEBQwAAAAAAAwFLAAAAAAADAUsA=', 'dtype': 'f8'},
              'yaxis': 'y'},
             {'hoverinfo': 'text',
              'marker': {'color': 'rgb(61,153,112)'},
              'mode': 'lines',
              'text': [opus_claude opus_asl_claude_as, , , ai
                       governance_governance_policymaking_career_roles],
              'type': 'scatter',
              'x': {'bdata': 'anMyCXbp7T/zMvwv6zDuP/My/C/rMO4/2nJ0V2Ta7T8=', 'dtype': 'f8'},
              'xaxis': 'x',
              'y': {'bdata': 'AAAAAABARcAAAAAAAEBFwAAAAAAAgFHAAAAAAACAUcA=', 'dtype': 'f8'},
              'yaxis': 'y'},
             {'hoverinfo': 'text',
              'marker': {'color': 'rgb(61,153,112)'},
              'mode': 'lines',
              'text': [opus_claude opus_claude_research_as, , , ai
                       2027_2027_timelines_years_time horizons],
              'type': 'scatter',
              'x': {'bdata': '8zL8L+sw7j8oOs73DVLuPyg6zvcNUu4/AAAAAAAAAAA=', 'dtype': 'f8'},
              'xaxis': 'x',
              'y': {'bdata': 'AAAAAAAgTMAAAAAAACBMwAAAAAAAQFXAAAAAAABAVcA=', 'dtype': 'f8'},
              'yaxis': 'y'},
             {'hoverinfo': 'text',
              'marker': {'color': 'rgb(61,153,112)'},
              'mode': 'lines',
              'text': [agents_alignment_agent_closest idea_closest idea to, , ,
                       opus_claude opus_claude_research_as],
              'type': 'scatter',
              'x': {'bdata': 'AAAAAAAAAAAjQtXz+XfuPyNC1fP5d+4/KDrO9w1S7j8=', 'dtype': 'f8'},
              'xaxis': 'x',
              'y': {'bdata': 'AAAAAAAAOcAAAAAAAAA5wAAAAAAAqFHAAAAAAACoUcA=', 'dtype': 'f8'},
              'yaxis': 'y'},
             {'hoverinfo': 'text',
              'marker': {'color': 'rgb(61,153,112)'},
              'mode': 'lines',
              'text': [in person program_the program in perso

In [12]:
topic_model.visualize_topics()

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'customdata': array([[0, 'regular | face | mjxc | bold | opentype', 30],
                                   [1, 'ai 2027 | 2027 | timelines | years | time horizons', 18],
                                   [2, 'model evals | evals | the evals | eval | spam', 16],
                                   [3, 'opus | claude opus | asl | claude | as', 16],
                                   [4, 'ai safety | safety | dangers | an ai | ai', 14],
                                   [5, 'features | networks | input | rank | things', 13],
                                   [6, 'thought | potential benefits | find myself | stress | potential',
                                    11],
                                   [7, 'control | control over | agi | ai | robots', 11],
                                   [8,
                                    'ai governance | governance | compute governance | policymaking | policymakers',
                                    10],
                                   [9, 'agents | alignment | agent | closest idea | closest idea to', 10],
                                   [10,
                                    'going to use mommunes | mommunes | be named | to be named | naming',
                                    10],
                                   [11,
                                    'advocacy efforts | advocacy | ai safety advocacy | alignment | safety advocacy',
                                    9],
                                   [12,
                                    'of simulators | simulators | simulator | the simulator | from simulator perspective',
                                    8],
                                   [13,
                                    'untrusted editing | trusted editing | untrusted editing protocols | the untrusted model | untrusted model',
                                    6],
                                   [14,
                                    'ai | national ai | beneficial ai | beneficial ai research | intelligence ai research',
                                    5],
                                   [15, 'life | living | life but | civilisation | of life', 4],
                                   [16, 'llms | llms are | the llms | llm | metacognitive', 4],
                                   [17, 'compliance costs | the senate | senate | regulations | senators',
                                    3],
                                   [18, 'video generation | audio | videos | ai videos | video and', 3],
                                   [19,
                                    'in person program | the program in person | program in person | person program | program in',
                                    3]], dtype=object),
              'hovertemplate': '<b>Topic %{customdata[0]}</b><br>%{customdata[1]}<br>Size: %{customdata[2]}',
              'legendgroup': '',
              'marker': {'color': '#B0BEC5',
                         'line': {'color': 'DarkSlateGrey', 'width': 2},
                         'size': {'bdata': 'HhIQEA4NCwsKCgoJCAYFBAQDAwM=', 'dtype': 'i1'},
                         'sizemode': 'area',
                         'sizeref': 0.01875,
                         'symbol': 'circle'},
              'mode': 'markers',
              'name': '',
              'orientation': 'v',
              'showlegend': False,
              'type': 'scatter',
              'x': {'bdata': ('RjWhQMlzq0BFFI9AipNfQXRkj0C52o' ... 'NADSGiQPbrf0BVd8FA3v5dQYxihUA='),
                    'dtype': 'f4'},
              'xaxis': 'x',
              'y': {'bdata': ('3Lb5PnXIk0CnQig/Qz6gQT2FjEBGeV' ... 'lAjl2cQA95v75uun9A53OfQb/2kEA='),
                    'dtype': 'f4'},
              'yaxis': 'y'}],
    'layout': {'annotations': [{'showarrow': False,
                                'text': 'D1',
                                'x': np.float32(3.0055199),
                                'y': np.float32(10.970647),
                

In [13]:
topic_model.visualize_topics(use_ctfidf=True)

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'customdata': array([[0, 'regular | face | mjxc | bold | opentype', 30],
                                   [1, 'ai 2027 | 2027 | timelines | years | time horizons', 18],
                                   [2, 'model evals | evals | the evals | eval | spam', 16],
                                   [3, 'opus | claude opus | asl | claude | as', 16],
                                   [4, 'ai safety | safety | dangers | an ai | ai', 14],
                                   [5, 'features | networks | input | rank | things', 13],
                                   [6, 'thought | potential benefits | find myself | stress | potential',
                                    11],
                                   [7, 'control | control over | agi | ai | robots', 11],
                                   [8,
                                    'ai governance | governance | compute governance | policymaking | policymakers',
                                    10],
                                   [9, 'agents | alignment | agent | closest idea | closest idea to', 10],
                                   [10,
                                    'going to use mommunes | mommunes | be named | to be named | naming',
                                    10],
                                   [11,
                                    'advocacy efforts | advocacy | ai safety advocacy | alignment | safety advocacy',
                                    9],
                                   [12,
                                    'of simulators | simulators | simulator | the simulator | from simulator perspective',
                                    8],
                                   [13,
                                    'untrusted editing | trusted editing | untrusted editing protocols | the untrusted model | untrusted model',
                                    6],
                                   [14,
                                    'ai | national ai | beneficial ai | beneficial ai research | intelligence ai research',
                                    5],
                                   [15, 'life | living | life but | civilisation | of life', 4],
                                   [16, 'llms | llms are | the llms | llm | metacognitive', 4],
                                   [17, 'compliance costs | the senate | senate | regulations | senators',
                                    3],
                                   [18, 'video generation | audio | videos | ai videos | video and', 3],
                                   [19,
                                    'in person program | the program in person | program in person | person program | program in',
                                    3]], dtype=object),
              'hovertemplate': '<b>Topic %{customdata[0]}</b><br>%{customdata[1]}<br>Size: %{customdata[2]}',
              'legendgroup': '',
              'marker': {'color': '#B0BEC5',
                         'line': {'color': 'DarkSlateGrey', 'width': 2},
                         'size': {'bdata': 'HhIQEA4NCwsKCgoJCAYFBAQDAwM=', 'dtype': 'i1'},
                         'sizemode': 'area',
                         'sizeref': 0.01875,
                         'symbol': 'circle'},
              'mode': 'markers',
              'name': '',
              'orientation': 'v',
              'showlegend': False,
              'type': 'scatter',
              'x': {'bdata': ('74EbQYNqTkHZnyJAYhtvQTYSKkFEZW' ... 'JBpiYoQS9ZEkBF81ZBgjAyQXDZH0E='),
                    'dtype': 'f4'},
              'xaxis': 'x',
              'y': {'bdata': ('ecUqwA/E5UHpsN5A7ODtQWDhKsDV7O' ... 'XA9StHwFQo10BU0OBBwUI0wAG0X8A='),
                    'dtype': 'f4'},
              'yaxis': 'y'}],
    'layout': {'annotations': [{'showarrow': False,
                                'text': 'D1',
                                'x': np.float32(1.1998967),
                                'y': np.float32(15.339445),
                

In [14]:
topic_model.visualize_heatmap()

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'coloraxis': 'coloraxis',
              'hovertemplate': 'x: %{x}<br>y: %{y}<br>Similarity Score: %{z}<extra></extra>',
              'name': '0',
              'type': 'heatmap',
              'x': [0_regular_face_mjxc, 1_ai 2027_2027_timelines, 2_model
                    evals_evals_the evals, 3_opus_claude opus_asl, 4_ai
                    safety_safety_dangers, 5_features_networks_input,
                    6_thought_potential benefit..., 7_control_control over_agi,
                    8_ai governance_governance_..., 9_agents_alignment_agent,
                    10_going to use mommunes_mo..., 11_advocacy efforts_advocac...,
                    12_of simulators_simulators..., 13_untrusted editing_truste...,
                    14_ai_national ai_beneficia..., 15_life_living_life but,
                    16_llms_llms are_the llms, 17_compliance costs_the sen...,
                    18_video generation_audio_v..., 19_in person program_the pr...],
              'xaxis': 'x',
              'y': [0_regular_face_mjxc, 1_ai 2027_2027_timelines, 2_model
                    evals_evals_the evals, 3_opus_claude opus_asl, 4_ai
                    safety_safety_dangers, 5_features_networks_input,
                    6_thought_potential benefit..., 7_control_control over_agi,
                    8_ai governance_governance_..., 9_agents_alignment_agent,
                    10_going to use mommunes_mo..., 11_advocacy efforts_advocac...,
                    12_of simulators_simulators..., 13_untrusted editing_truste...,
                    14_ai_national ai_beneficia..., 15_life_living_life but,
                    16_llms_llms are_the llms, 17_compliance costs_the sen...,
                    18_video generation_audio_v..., 19_in person program_the pr...],
              'yaxis': 'y',
              'z': {'bdata': ('BAAAAAAA8D9H1HwS9bPOP1FAvzur29' ... 'wEeD2tP2zb+ezCfso/BQAAAAAA8D8='),
                    'dtype': 'f8',
                    'shape': '20, 20'}}],
    'layout': {'coloraxis': {'colorbar': {'title': {'text': 'Similarity Score'}},
                             'colorscale': [[0.0, 'rgb(247,252,240)'], [0.125,
                                            'rgb(224,243,219)'], [0.25,
                                            'rgb(204,235,197)'], [0.375,
                                            'rgb(168,221,181)'], [0.5,
                                            'rgb(123,204,196)'], [0.625,
                                            'rgb(78,179,211)'], [0.75,
                                            'rgb(43,140,190)'], [0.875,
                                            'rgb(8,104,172)'], [1.0,
                                            'rgb(8,64,129)']]},
               'height': 800,
               'hoverlabel': {'bgcolor': 'white', 'font': {'family': 'Rockwell', 'size': 16}},
               'legend': {'title': {'text': 'Trend'}},
               'margin': {'t': 60},
               'showlegend': True,
               'template': '...',
               'title': {'font': {'color': 'Black', 'size': 22},
                         'text': '<b>Similarity Matrix</b>',
                         'x': 0.55,
                         'xanchor': 'center',
                         'y': 0.95,
                         'yanchor': 'top'},
               'width': 800,
               'xaxis': {'anchor': 'y', 'constrain': 'domain', 'domain': [0.0, 1.0], 'scaleanchor': 'y'},
               'yaxis': {'anchor': 'x', 'autorange': 'reversed', 'constrain': 'domain', 'domain': [0.0, 1.0]}}
})

In [16]:
# let's save the model 
model_path = os.path.join(os.getcwd(), "bert_topic_model.pkl")
topic_model.save(model_path, "pickle")

2025-06-18 18:11:39,040 - BERTopic - WARNING: When you use `pickle` to save/load a BERTopic model,please make sure that the environments in which you saveand load the model are **exactly** the same. The version of BERTopic,its dependencies, and python need to remain the same.


In [17]:
# Example loading and using the model to predict a new post
model = BERTopic.load(model_path)

pred_topics, pred_probs = model.transform("This random sentence is about steering and embeddings and I'll also mention vectors. Let's see if it gets classified into the right topic.")

Batches: 100%|██████████| 1/1 [00:00<00:00,  3.85it/s]
2025-06-18 18:11:47,911 - BERTopic - Dimensionality - Reducing dimensionality of input embeddings.
2025-06-18 18:11:52,264 - BERTopic - Dimensionality - Completed ✓
2025-06-18 18:11:52,265 - BERTopic - Clustering - Approximating new points with `hdbscan_model`
2025-06-18 18:11:52,274 - BERTopic - Cluster - Completed ✓


In [18]:
topic_model.get_topic(pred_topics[0].item())

[('features', np.float32(0.36850584)),
 ('networks', np.float32(0.28428304)),
 ('input', np.float32(0.2673971)),
 ('rank', np.float32(0.25791323)),
 ('things', np.float32(0.25773397)),
 ('matrix', np.float32(0.25429153)),
 ('what', np.float32(0.24983431)),
 ('get', np.float32(0.2443426)),
 ('components', np.float32(0.240389)),
 ('network', np.float32(0.23296337))]